# Recipe 10: ENAS with Evolution

This notebook implements ENAS combined with evolutionary algorithms for architecture search.

## Imports and Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Define constants
NUM_OPS = 5
NUM_NODES = 4
POPULATION_SIZE = 100
TOURNAMENT_SIZE = 5
MUTATION_RATE = 0.1

## Network and Cell Definitions

In [ ]:
class MixedOp(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([
            nn.Conv2d(C, C, 3, padding=1),
            nn.Conv2d(C, C, 5, padding=2),
            nn.Conv2d(C, C, 3, padding=1, groups=C),
            nn.Conv2d(C, C, 5, padding=2, groups=C),
            nn.Identity()
        ])

    def forward(self, x, choice):
        return self.ops[choice](x)

class Cell(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.nodes = nn.ModuleList([MixedOp(C) for _ in range(NUM_NODES)])

    def forward(self, x, actions):
        states = [x]
        for node, (op1, op2, combine) in zip(self.nodes, actions):
            s1 = node(states[op1], combine[0])
            s2 = node(states[op2], combine[1])
            states.append(s1 + s2)
        return states[-1]

class Network(nn.Module):
    def __init__(self, C, num_classes, num_cells):
        super().__init__()
        self.stem = nn.Conv2d(3, C, 3, padding=1)
        self.cells = nn.ModuleList([Cell(C) for _ in range(num_cells)])
        self.classifier = nn.Linear(C, num_classes)

    def forward(self, x, actions):
        x = self.stem(x)
        for cell, cell_action in zip(self.cells, actions):
            x = cell(x, cell_action)
        x = x.mean([2, 3])
        return self.classifier(x)

## Evolutionary ENAS

In [ ]:
class Individual:
    def __init__(self, num_cells):
        self.genome = [self.random_cell() for _ in range(num_cells)]
        self.fitness = 0

    @staticmethod
    def random_cell():
        return [
            (random.randint(0, NUM_OPS-1), random.randint(0, NUM_OPS-1), random.randint(0, 1))
            for _ in range(NUM_NODES)
        ]

    def mutate(self):
        for cell in self.genome:
            for node in cell:
                if random.random() < MUTATION_RATE:
                    idx = random.randint(0, 2)
                    if idx < 2:
                        node[idx] = random.randint(0, NUM_OPS-1)
                    else:
                        node[idx] = random.randint(0, 1)

def crossover(parent1, parent2):
    child = Individual(len(parent1.genome))
    for i in range(len(child.genome)):
        if random.random() < 0.5:
            child.genome[i] = parent1.genome[i]
        else:
            child.genome[i] = parent2.genome[i]
    return child

def tournament_selection(population):
    tournament = random.sample(population, TOURNAMENT_SIZE)
    return max(tournament, key=lambda ind: ind.fitness)

def train_evolutionary_enas(network, train_data, val_data, num_generations):
    population = [Individual(len(network.cells)) for _ in range(POPULATION_SIZE)]

    for generation in range(num_generations):
        # Evaluate population
        for ind in population:
            ind.fitness = evaluate(network, ind.genome, val_data)

        # Select best individual
        best_ind = max(population, key=lambda ind: ind.fitness)
        print(f"Generation {generation}, Best Fitness: {best_ind.fitness:.4f}")

        # Create new population
        new_population = [best_ind]  # Elitism
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            child.mutate()
            new_population.append(child)

        population = new_population

        # Train network with best architecture
        train_network(network, best_ind.genome, train_data)

    return best_ind.genome

def train_network(network, actions, train_data):
    optimizer = optim.Adam(network.parameters(), lr=0.01)
    network.train()
    for x, y in train_data:
        optimizer.zero_grad()
        loss = nn.CrossEntropyLoss()(network(x, actions), y)
        loss.backward()
        optimizer.step()

def evaluate(model, actions, data):
    model.eval()
    correct = 0
    total = 0
    for x, y in data:
        with torch.no_grad():
            outputs = model(x, actions)
            _, predicted = outputs.max(1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return correct / total

## Usage Example

In [ ]:
# Assuming you have your data loaded as train_data and val_data
network = Network(16, 10, 8)  # 16 channels, 10 classes, 8 cells
# Uncomment the following line to train
# best_architecture = train_evolutionary_enas(network, train_data, val_data, num_generations=50)